# <img style="float: left; padding: 0px 10px 0px 0px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg/1920px-Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg.png"  width="80" /> MCD3100 - Ciencia de Datos Geoespaciales
**Pontificia Universidad Católica de Chile**<br>
**Magister en Ciencia de Datos**<br>

# Tutorial 2: Datos vectoriales I.


Este tutorial tiene como objetivos:
- Realizar calculos geométricos simples.
- Construir nuevas geometrías mediantes buffers y uniones.
- Realizar operaciones de superposición y unión espacial de capas vectoriales.
- Elaborar gráficos de coropletas.


In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely
import contextily as ctx

## 2. Datos.

Para este tutorial utilizaremos los datos exportados en el Tutorial N°1:
- `vm_comuna.gpkg`
- `vm_vial.gpkg`
- `vm_manz.gpkg`

Puedes usar los mismos datos que generaste en el Tutorial 1, o usar los adjuntos a este tutorial.

Agregaremos además los datos de población del Censo 2017 (ya que los resultados de población por manzana censal del Censo 2024 aún no están disponibles):

- `vm_censo2017.gpkg`

Y también, una capa disponible en la IDE Chile con la geometría de la [línea litoral de Chile](https://www.geoportal.cl/geoportal/catalog/35650/L%C3%ADnea%20Litoral):`LneaLitoral.shp`








## 3. Ejercicios.

En este tutorial, buscaremos respondar a la siguientes preguntas:
- ¿Cuál es la superficie y largo total de calles de Viña del Mar?
- ¿Qué manzanas de la comuna están ubicadas a menos de 500 m de la línea de playa? Este es un dato relevante por ejemplo para identificar ubicaciones de interés para propiedades en arriendo, o para analizar la correlación entre precios de propiedades y cercanía a la playa.
- ¿Cuántas personas viven a menos de 500 m de la línea de playa? Este es un dato relevante por ejemplo para la planificación de medidas de evacuación en caso de tsunami.

### 3.1 Lectura e inspección de datos vectoriales.

In [ ]:
vina=gpd.read_file('vm_comuna.gpkg')
vial=gpd.read_file('vm_vial.gpkg')
manz=gpd.read_file('vm_manz.gpkg')
censo=gpd.read_file('vm_censo2017.gpkg')

In [ ]:
vina.crs

In [ ]:
litoral=gpd.read_file('Costa/LneaLitoral.shp')
litoral.crs

In [ ]:
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111)

crs='3857'

vina.to_crs(crs).boundary.plot(ax=ax,color='navy',lw=2,label='Comuna de Viña del Mar')
vial.to_crs(crs).plot(ax=ax,color='k',lw=0.5,label='Ejes viales')
manz.to_crs(crs).boundary.plot(ax=ax,lw=0.2,label='Manzanas')
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
ax.legend();
plt.show()

### 3.2 ¿Cuál es la superficie de Viña del Mar?

In [ ]:
vina.area  #en grados sexagesimales

In [ ]:
vina.to_crs(32719).area #en metros cuadrados

In [ ]:
vina.to_crs(32719).area/1e6 #en km cuadrados

### 3.3 ¿Cuál es el largo total de calles en la comuna?

In [ ]:
vial.head()

In [ ]:
vial.to_crs(32719).length.sum()/1000

### 3.4 Línea de costa de la comuna.

Dado que trabajaremos con distancias en metros, reproyectamos los datos al sistema UTM (EPSG:32719).

In [ ]:
litoral=litoral.to_crs(32719)
litoral.plot()

La capa con la línea de playa contiene información para todo Chile, pero para efectos de este ejercicio sólo nos interesa la comuna de Viña del Mar. Por lo tanto, utilizamos la función `overlay` para crear un nuevo GeoDataFrame con la intersección de ambas capas:

In [ ]:
vina=vina.to_crs(32719)
cvm=litoral.overlay(vina)
cvm.plot();

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)

crs='3857' #usamos este SRC para graficar y agregar el mapa base con contextily

vina.to_crs(crs).boundary.plot(ax=ax,color='navy',lw=2,label='Comuna de Viña del Mar')
cvm.to_crs(crs).plot(ax=ax,color='y',lw=2,label='Playas')
#area.to_crs(crs).plot(ax=ax,color='y',alpha=0.5,label='Playas')
manz.to_crs(crs).boundary.plot(ax=ax,lw=0.2,label='Manzanas')
#manz_beach.to_crs(crs).boundary.plot(ax=ax,color='r',lw=0.2,label='Manzanas cerca playa')
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
ax.legend();
plt.show()

### 3.5 ¿Qué manzanas se encuentran a menos de 500 m de la playa?

Creamos un buffer de 500m alrededor de la línea litoral:

In [ ]:
buf=cvm.buffer(500)
buf.boundary.plot()

Combinamos los buffers individuales en una única geometría usando `unary_union`, y usamos esta geometría para crear un nuevo GeoDataFrame:

In [ ]:
area=gpd.GeoDataFrame([buf.unary_union], columns=['geometry'],crs=32719)
area.boundary.plot()

In [ ]:
area

In [ ]:
manz_beach=manz.to_crs(32719).overlay(area)

In [ ]:
manz_beach.plot()

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)

crs='3857' #usamos este SRC para graficar y agregar el mapa base con contextily

vina.to_crs(crs).boundary.plot(ax=ax,color='navy',lw=2,label='Comuna de Viña del Mar')
cvm.to_crs(crs).plot(ax=ax,color='y',lw=2,label='Playas')
manz.to_crs(crs).boundary.plot(ax=ax,lw=0.2,label='Manzanas')
manz_beach.to_crs(crs).boundary.plot(ax=ax,color='r',lw=0.2,label='Manzanas cerca playa')
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
ax.legend();
plt.show()

### 3.6 ¿Cuánta población habita en zonas a 500 m de la línea litoral?

In [ ]:
censo2017=gpd.read_file('vm_censo2017.gpkg').to_crs(32719)
censo2017.columns

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)

crs='3857' #usamos este SRC para graficar y agregar el mapa base con contextily

vina.to_crs(crs).boundary.plot(ax=ax,color='navy',lw=2,label='Comuna de Viña del Mar')
cvm.to_crs(crs).plot(ax=ax,color='y',lw=2,label='Playas')
manz.to_crs(crs).boundary.plot(ax=ax,lw=0.2,label='Manzanas')
manz_beach.to_crs(crs).boundary.plot(ax=ax,color='r',lw=0.2,label='Manzanas cerca playa')
censo2017.to_crs(crs).plot(ax=ax,markersize=3,color='k')
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
ax.legend();
plt.show()

In [ ]:
pob_costa=gpd.sjoin(manz_beach,censo2017[['TOTAL_PERS','geometry']],how='left')
pob_costa.head()

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)

crs='3857'

#vina.to_crs(crs).boundary.plot(ax=ax,color='navy',lw=2,label='Comuna de Viña del Mar')
area.to_crs(crs).boundary.plot(ax=ax,color='y',alpha=0.5,label='Litoral - 500 m')
#manz.to_crs(crs).boundary.plot(ax=ax,lw=0.2,label='Manzanas')
manz_beach.to_crs(crs).boundary.plot(ax=ax,color='r',lw=0.2,label='Manzanas cerca playa')
pob_costa.to_crs(crs).plot(ax=ax,cmap='Reds',column='TOTAL_PERS',label='Población')
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
ax.legend();
plt.show()

In [ ]:
pob_costa.TOTAL_PERS.sum()